In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, create_optimizer
from sklearn.model_selection import train_test_split

In [ ]:
# Data dummy
texts = ["I love this!", "I hate this!", "This is okay."]
labels = [1, 0, 1]  # 1: Positive, 0: Negative

# Tokenizer dan encoding
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_examples(texts, labels):
    encodings = tokenizer(texts, padding='max_length', truncation=True, return_tensors='tf')
    dataset = tf.data.Dataset.from_tensor_slices((
        dict(encodings),
        labels
    ))
    return dataset

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2)
train_dataset = encode_examples(train_texts, train_labels)
test_dataset = encode_examples(test_texts, test_labels)

In [ ]:
# Model BERT
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Pelatihan
num_train_steps = len(train_dataset) * 3
optimizer, _ = create_optimizer(init_lr=2e-5, num_train_steps=num_train_steps, num_warmup_steps=0)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

In [ ]:
model.fit(train_dataset.batch(8), epochs=3, validation_data=test_dataset.batch(8))

In [ ]:
# Evaluasi
eval_results = model.evaluate(test_dataset.batch(8))
print(eval_results)

# Prediksi
inputs = tokenizer("I really like this!", return_tensors='tf')
outputs = model(inputs)
logits = outputs.logits
predicted_class_id = tf.argmax(logits, axis=-1).numpy()[0]
print(predicted_class_id)